In [1]:
import rootutils

rootutils.setup_root("./", indicator=".project-root", pythonpath=True)

%load_ext autoreload
%autoreload 2

import torch
import torch_geometric
from topobenchmarkx.data.datasets import CustomDataset
import hydra
from hydra import initialize, compose
from topobenchmarkx.data.dataloader_fullbatch import FullBatchDataModule
from topobenchmarkx.data.load.loaders import (
    GraphLoader,
    SimplicialLoader,
    HypergraphLoader,
)

initialize(config_path="../configs", job_name="job")
config = compose(config_name="train.yaml", return_hydra_config=True)

/tmp/ipykernel_418258/724119490.py:20: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  initialize(config_path="../configs", job_name="job")


In [2]:
config.dataset.transforms

{'_target_': 'topobenchmarkx.transforms.transformchain.TransformChain', 'transform1': {'_target_': 'topobenchmarkx.transforms.lifting.DataLiftingTransform', 'type': 'Null Lifting (Identity)', 'lifting': 'Identity'}, 'transform2': {'_target_': 'topobenchmarkx.transforms.lifting.DataLiftingTransform', 'lifting': 'HypergraphKHopLifting', 'k_value': 1}}

In [3]:
a = hydra.utils.instantiate(config.dataset, _recursive_=False)
dataset = a.load()

Transform parameters are the same, using existing data_dir: /TopoBenchmarkX/datasets/graph/cocitation/Cora/transform1_transform2_/2303687994


/usr/local/lib/python3.11/site-packages/torch_geometric/data/in_memory_dataset.py:290: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)


In [4]:
dataset[0]

([tensor(indices=tensor([[   0,    0,    0,  ..., 2707, 2707, 2707],
                         [   0,  633, 1862,  ..., 1473, 2706, 2707]]),
         values=tensor([1., 1., 1.,  ..., 1., 1., 1.]),
         size=(2708, 2708), nnz=13264, layout=torch.sparse_coo),
  tensor([1890,  790, 2458, 2352,  366, 2543, 1815, 2692, 2541,  228,  912, 2179,
           417, 1230,  742,   79, 1214,  492, 1909, 1169, 1342,  824, 2240, 2151,
          1481, 2574, 2405, 1024,  592, 2301, 2524, 2445, 1797, 2101,  127, 2297,
           403,  150, 2614, 1437, 1629,   62,   95,  959, 2122, 1576, 2132,  236,
           222, 2542, 1810,  120, 1685,  193,  181,  389,  968, 2219, 1545,  669,
          1365, 1394,  331,  614, 1286,  266, 1932, 1807,  225,  105, 1508,  630,
          2068,  870, 2098, 1186, 1891, 1755, 1926,  707, 2288, 1688,  284, 1659,
           920, 1433, 2328, 2191,  734, 1812, 2681,  922, 1402, 2548,  167, 2519,
            12, 1546,  617, 1092,   90,  612, 2453, 1404,  857, 2233,  777, 1638,
 

Processing...
Done!
/usr/local/lib/python3.11/site-packages/torch_geometric/data/in_memory_dataset.py:290: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)


In [7]:
dataset[0]

([tensor([1363,  178,  498,  340, 1359, 1884, 2442,  198, 2550, 2085, 1503, 1166,
          1848, 2671, 2663, 1339, 2201,  759,  170, 1356,  773, 2326, 2259,  992,
            45, 1689,  467, 1759, 2155, 1977, 1902,  319,  184, 1739, 1538, 1673,
          1450, 1509,  839, 1540, 2269,  744,  539, 1154,  801, 2307, 1135,  210,
          1621, 1639,   65,  863, 1632, 1651,  203, 1132, 1021, 2033, 1026, 2696,
           341, 2086,  875, 1120,  372,  576,  942,  211,  520,  939,  889, 1258,
          2341, 1591, 1998, 1325,  354,   96, 2477,  589, 2683, 2497, 2367,  513,
            71, 1577, 2518, 2109, 1895,  456,  725,   68, 1525, 1917, 1458,  881,
          1581,  231,  377, 2196,  862,  432, 1460, 2506,  994, 1858, 1559, 1378,
          1143, 1758,  172, 2250, 1248,  814,  208,  218,  948,   16, 2054,  367,
          1406, 1128,  884, 1319, 1388, 2422, 2529, 1567, 2366, 2210, 1573,  394,
          1921,  984,  542,  803,  677,  406, 1459,  552,  362,  549,  970, 2143,
          1900, 

In [2]:
"transforms" in config.dataset

True

# Load data

In [3]:
data_loader = GraphLoader(config.dataset)
data = data_loader.load()

Transform parameters are the same, using existing data_dir: /Users/gbg141/Documents/TopoProjectX/TopoBenchmarkX/datasets/graph/cocitation/Cora/HypergraphKHopLifting/885927755


/Users/gbg141/Documents/TopoProjectX/TopoBenchmarkX/venv_topox/lib/python3.11/site-packages/torch_geometric/data/in_memory_dataset.py:293: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)


In [4]:
data

Data(x=[2708, 1433], y=[2708], incidence_1=[2708, 2708], num_hyperedges=2708, x_0=[2708, 1433], x_hyperedges=[2708, 1433])

In [28]:
simplicial_loader = SimplicialLoader(config.dataset)
data = simplicial_loader.load()

In [34]:
data.

{}

In [22]:
import toponetx
import numpy as np

In [23]:
shrec, _ = toponetx.datasets.mesh.shrec_16(size="small")
shrec = {key: np.array(value) for key, value in shrec.items()}

Loading shrec 16 small dataset...

done!


In [27]:
print(shrec["complexes"][0])

Simplicial Complex with shape (252, 750, 500) and dimension 2


### Lifting

In [5]:
# # Load data
# from topobenchmarkx.data.load.loaders import HypergraphLoader

# data_loader = HypergraphLoader(config)
# data = data_loader.load()

In [4]:
config.keys()

dict_keys(['hydra', 'task_name', 'tags', 'train', 'test', 'ckpt_path', 'seed', 'dataset', 'transforms', 'model', 'evaluator', 'callbacks', 'trainer', 'paths', 'extras'])

In [5]:
config["transforms"]

{'_target_': 'topobenchmarkx.transforms.lifting.DataLiftingTransform', 'lifting': 'HypergraphKHopLifting', 'k_value': 1, 'complex_dim': 'None', 'max_triangles': 'None', 'aggregation_method': 'None'}

In [6]:
lifting = hydra.utils.instantiate(config.transforms)

In [7]:
lifting.parameters

{'k_value': 1,
 'complex_dim': 'None',
 'max_triangles': 'None',
 'aggregation_method': 'None',
 'lifting': 'HypergraphKHopLifting'}

In [8]:
data = torch_geometric.data.Data()
data.x = torch.zeros([6, 1])
data.edge_index = torch.tensor([[0, 0, 0, 1, 1, 1, 2, 4], [1, 2, 3, 2, 3, 4, 3, 5]])

In [9]:
lifted_data = lifting(data)

In [10]:
lifted_data

Data(x=[6, 1], incidence_1=[6, 6], num_hyperedges=6, x_0=[6, 1], x_hyperedges=[6, 1])

In [ ]:
cora = torch_geometric.datasets.Planetoid(
    root="../datasets/graph/", name="Cora", pre_transform=lifting
)

/Users/gbg141/Documents/TopoProjectX/TopoBenchmarkX/venv_topox/lib/python3.11/site-packages/torch_geometric/data/dataset.py:234: UserWarning: The `pre_transform` argument differs from the one used in the pre-processed version of this dataset. If you want to make use of another pre-processing technique, pass `force_reload=True` explicitly to reload the dataset.
  warnings.warn(


In [ ]:
cora.data

/Users/gbg141/Documents/TopoProjectX/TopoBenchmarkX/venv_topox/lib/python3.11/site-packages/torch_geometric/data/in_memory_dataset.py:293: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)


Data(x=[2708, 1433], num_cells_0=2708, num_cells_1=10556, num_cells_2=2648, incidence_1=[2708, 10556], incidence_2=[10556, 2648], laplacian_up_1=[10556, 10556], laplacian_up_2=[2648, 2648], laplacian_down_2=[10556, 10556], laplacian_down_1=[2708, 2708], x_0=[2708, 1433], x_1=[10556, 1433], x_2=[2648, 1433])

In [ ]:
from topobenchmarkx.data.datasets import CustomDataset

In [ ]:
from torch.utils.data import DataLoader
from torch_geometric.data import Data


def collate_fn(batch):
    """
    args:
        batch - list of (tensor, label)

    reutrn:
        xs - a tensor of all examples in 'batch' after padding
        ys - a LongTensor of all labels in batch
    """

    for b in batch:
        values, keys = b[0], b[1]
        data = Data()
        for key, value in zip(keys, values):
            data[key] = value

    return data


d = DataLoader(dataset=CustomDataset([cora.data]), batch_size=1, collate_fn=collate_fn)
next(iter(d))

Data(num_hyperedges=2708, x_0=[2708, 1433], incidence_1=[2708, 2708], x=[2708, 1433], x_hyperedges=[2708, 1433])

In [ ]:
databatch = torch_geometric.data.Batch.from_data_list(data_lst)

NameError: name 'data_lst' is not defined

In [ ]:
databatch.edge_index[:, :4905]

tensor([[   0,    1,    2,  ..., 2706, 2706, 2707],
        [ 978,  736,  399,  ...,  362,  419,  921]])

In [ ]:
databatch.edge_index[:, 4905:]

tensor([[2708, 2709, 2710,  ..., 5414, 5414, 5415],
        [3686, 3444, 3107,  ..., 3070, 3127, 3629]])

In [ ]:
databatch.edge_index = databatch.edge_index.to_sparse()
databatch.edge_index

AllSetTransformer(
  (layers): ModuleList(
    (0): AllSetTransformerLayer(
      (vertex2edge): AllSetTransformerBlock(
        (multihead_att): MultiHeadAttention()
        (mlp): MLP(
          (0): Linear(in_features=64, out_features=64, bias=False)
          (1): Dropout(p=0.2, inplace=False)
        )
        (ln0): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (ln1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      )
      (edge2vertex): AllSetTransformerBlock(
        (multihead_att): MultiHeadAttention()
        (mlp): MLP(
          (0): Linear(in_features=64, out_features=64, bias=False)
          (1): Dropout(p=0.2, inplace=False)
        )
        (ln0): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (ln1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      )
    )
  )
)

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader


class TextDataset(Dataset):
    def __init__(self, text_list, labels):
        self.text_list = text_list
        self.labels = labels

    def __len__(self):
        return len(self.text_list)

    def __getitem__(self, idx):
        text = self.text_list[idx]
        label = self.labels[idx]

        # You can perform any text preprocessing here if needed
        # For example, tokenization, numerical encoding, etc.

        return {"text": text, "label": label}


# Example usage
text_data = [
    "This is an example.",
    "Another text sample.",
    "PyTorch DataLoader with text.",
]
labels = [0, 1, 0]  # Example labels

# Create an instance of your custom dataset
custom_dataset = TextDataset(text_data, labels)

# Use DataLoader to load batches of data
batch_size = 2
data_loader = DataLoader(custom_dataset, batch_size=batch_size, shuffle=True)

# Iterate through batches
for batch in data_loader:
    texts = batch["text"]
    labels = batch["label"]

    # Perform your training/validation/test operations here
    print("Texts:", texts)
    print("Labels:", labels)

In [ ]:
data_lst[0].keys()

In [ ]:
from torch.utils.data import DataLoader

dataloader = DataLoader(dataset, batch_size=1, shuffle=True)
next(iter(dataloader))

In [ ]:
from torch_geometric.loader import DataLoader

dataloader = DataLoader(dataset, batch_size=1, shuffle=False)

next(iter(dataloader))

In [ ]:
config.data

### Loss

In [ ]:
loss = hydra.utils.instantiate(config.model.loss)

### Backbone

In [ ]:
hydra.utils.instantiate(config.model.backbone)

### ReadOut

In [ ]:
readout = hydra.utils.instantiate(config.model.readout)

In [ ]:
readout.parameters()

<generator object Module.parameters at 0x19ca5db60>

### Evaluator

In [ ]:
evaluator = hydra.utils.instantiate(config.evaluator)

In [ ]:
import torch

d = {
    "labels": torch.tensor([0, 1, 2, 1, 2, 2]),
    "logits": torch.tensor(
        [[1, 11, 11], [5, 11, 3], [2, 3, 4], [5, 16, 7], [8, 9, 10], [11, 12, 13]]
    ),
}


evaluator.eval(d)

/Users/gbg141/Documents/TopoProjectX/TopoBenchmarkX/venv_topox/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'labels': tensor([0, 1, 2, 1, 2, 2]),
 'logits': tensor([[ 1, 11, 11],
         [ 5, 11,  3],
         [ 2,  3,  4],
         [ 5, 16,  7],
         [ 8,  9, 10],
         [11, 12, 13]]),
 'metrics': {'acc': 0.8333333333333334,
  'pre_micro': 0.8333333333333334,
  'pre_macro': 0.5555555555555555,
  'rec_micro': 0.8333333333333334,
  'rec_macro': 0.6666666666666666,
  'f1_micro': 0.8333333333333334,
  'f1_macro': 0.6}}

In [ ]:
d["logits"].argmax(dim=1)

tensor([1, 1, 2, 1, 2, 2])

### Dataloader

In [ ]:
# Load data
from topobenchmarkx.data.load.loaders import HypergraphLoader
from topobenchmarkx.data.dataloader_fullbatch import FullBatchDataModule

data_loader = HypergraphLoader(config)
data = data_loader.load()
dataloader = FullBatchDataModule(data=data)

Loading hypergraph dataset name: cora
number of nodes:2708, feature dimension: 1433
number of hyperedges: 1072
Final num_hyperedges 1392
Final num_nodes 2708
Final num_class 7


FileNotFoundError: [Errno 2] No such file or directory: '/Users/gbg141/Documents/TopoProjectX/TopoBenchmarkX/data//data_splits/cora/train_prop=0.5/split_0.npz'

In [ ]:
batch = next(iter(dataloader.train_dataloader()))

In [ ]:
batch.x[batch.train_mask]

In [ ]:
a = hydra.utils.instantiate(config.model)

In [ ]:
a.hparams.backbone is a.backbone

In [ ]:
id(a.hparams.backbone) == id(a.backbone)

In [ ]:
b = a.backbone.__class__

In [ ]:
b in []topomodelx.nn.hypergraph.unigcnii.UniGCNII

In [ ]:
import topomodelx